<a href="https://colab.research.google.com/github/sajithrw/RetinaInspect-AI/blob/develop/RetinaInspect_AI_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>